# Importation des bibliothèques nécessaires

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib
from sklearn.svm import SVC 

# import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

In [2]:
# Chargement des données
data = pd.read_csv("machine2.csv")
data.head()

,Site,Code site,Branche,Chargé d'exploitation,Energie,Semaine de l'alerte,Mois de l'alerte,Date d'envoi de l'alerte au client,Client alerté,Chargé d'analyse BO,...,Talon surconso identifié,DP Surconso (kW),Nb d'heures,Nb nuits/jours concernés,Impact conso (kWh),Type d'impact,% Surconso,Période d'alerte,Type de surconso,Catégorie
0,AIX EST,CS625,SES,DM,Electricité,23,6,,0,RC,...,110,10.0,5.0,6,300.0,Récurrent,0.1,Nuit,Impact Client,Froid
1,AIX EST,CS625,SES,DM,Electricité,24,6,,0,HH,...,115,15.0,5.0,7,525.0,Récurrent,0.15,Nuit,Impact Client,Froid
2,AIX EST,CS625,SES,DM,Electricité,22,6,,0,RC,...,120,20.0,5.0,3,300.0,Récurrent,0.2,Nuit,Impact Client,Froid
3,AIX EST,CS625,SES,DM,Electricité,25,6,,0,HH,...,120,20.0,5.0,7,700.0,Récurrent,0.2,Nuit,Impact Client,Froid
4,AIX EST,CS625,SES,DM,Electricité,26,6,,0,HH,...,120,20.0,5.0,6,600.0,Récurrent,0.2,Nuit,Impact Client,Froid


In [3]:
df = data[["Talon surconso identifié", "DP Surconso (kW)", "Nb d'heures", "Nb nuits/jours concernés", "Impact conso (kWh)", "% Surconso", "Période d'alerte", "Catégorie"]]
df.head(20)

,Talon surconso identifié,DP Surconso (kW),Nb d'heures,Nb nuits/jours concernés,Impact conso (kWh),% Surconso,Période d'alerte,Catégorie
0,110,10.0,5.0,6,300.0,0.1,Nuit,Froid
1,115,15.0,5.0,7,525.0,0.15,Nuit,Froid
2,120,20.0,5.0,3,300.0,0.2,Nuit,Froid
3,120,20.0,5.0,7,700.0,0.2,Nuit,Froid
4,120,20.0,5.0,6,600.0,0.2,Nuit,Froid
5,145,45.0,5.0,1,225.0,0.45,Nuit,Froid
6,165,65.0,9.0,2,1170.0,0.65,Dim,Changement de comportement
7,175,75.0,9.0,1,675.0,0.75,Dim,Changement de comportement
8,200,100.0,9.0,1,900.0,1,Dim,Changement de comportement
9,85,5.0,7.0,1,35.0,0.0625,Dim,Changement de comportement


In [4]:
print(df.dtypes)

Talon surconso identifié      int64
DP Surconso (kW)            float64
Nb d'heures                 float64
Nb nuits/jours concernés      int64
Impact conso (kWh)          float64
% Surconso                   object
Période d'alerte             object
Catégorie                    object
dtype: object


In [5]:
# Créer un dictionnaire de mapping
mapping = {"Investigation en cours": 0, "Investigation en cours ": 0, "Changement de comportement": 1, "Eclairage": 2, "Froid": 3, "CVC": 4, "Multi-usage": 5, " ": 6, "Process": 7}

# Remplacer les valeurs de la colonne 'Catégorie' par les entiers correspondants
df['Catégorie'] = df['Catégorie'].replace(mapping)

# Afficher les 20 premières lignes du DataFrame
df.head(20)

C:\Users\Lahoucine\AppData\Local\Temp\ipykernel_1848\1883936991.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Catégorie'] = df['Catégorie'].replace(mapping)


,Talon surconso identifié,DP Surconso (kW),Nb d'heures,Nb nuits/jours concernés,Impact conso (kWh),% Surconso,Période d'alerte,Catégorie
0,110,10.0,5.0,6,300.0,0.1,Nuit,3
1,115,15.0,5.0,7,525.0,0.15,Nuit,3
2,120,20.0,5.0,3,300.0,0.2,Nuit,3
3,120,20.0,5.0,7,700.0,0.2,Nuit,3
4,120,20.0,5.0,6,600.0,0.2,Nuit,3
5,145,45.0,5.0,1,225.0,0.45,Nuit,3
6,165,65.0,9.0,2,1170.0,0.65,Dim,1
7,175,75.0,9.0,1,675.0,0.75,Dim,1
8,200,100.0,9.0,1,900.0,1,Dim,1
9,85,5.0,7.0,1,35.0,0.0625,Dim,1


In [6]:
df['Catégorie'] = pd.to_numeric(df['Catégorie'], errors='coerce')
# Supprimer les lignes avec une valeur vide dans la colonne spécifique
df = df.dropna(subset=['Catégorie'])

# Réinitialiser les index du DataFrame
df = df.reset_index(drop=True)

# Afficher le DataFrame mis à jour
df.head(20)

C:\Users\Lahoucine\AppData\Local\Temp\ipykernel_1848\2459279336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Catégorie'] = pd.to_numeric(df['Catégorie'], errors='coerce')


,Talon surconso identifié,DP Surconso (kW),Nb d'heures,Nb nuits/jours concernés,Impact conso (kWh),% Surconso,Période d'alerte,Catégorie
0,110,10.0,5.0,6,300.0,0.1,Nuit,3.0
1,115,15.0,5.0,7,525.0,0.15,Nuit,3.0
2,120,20.0,5.0,3,300.0,0.2,Nuit,3.0
3,120,20.0,5.0,7,700.0,0.2,Nuit,3.0
4,120,20.0,5.0,6,600.0,0.2,Nuit,3.0
5,145,45.0,5.0,1,225.0,0.45,Nuit,3.0
6,165,65.0,9.0,2,1170.0,0.65,Dim,1.0
7,175,75.0,9.0,1,675.0,0.75,Dim,1.0
8,200,100.0,9.0,1,900.0,1,Dim,1.0
9,85,5.0,7.0,1,35.0,0.0625,Dim,1.0


In [7]:
# Créer un dictionnaire de mapping
mapping = {"Nuit": 0, "Dim": 1, "Jour": 2, "Jour/nuit": 3, "Jour/Nuit": 3}

# Remplacer les valeurs de la colonne 'Catégorie' par les entiers correspondants
df["Période d'alerte"] = df["Période d'alerte"].replace(mapping)

# Afficher les 20 premières lignes du DataFrame
df.head(20)

,Talon surconso identifié,DP Surconso (kW),Nb d'heures,Nb nuits/jours concernés,Impact conso (kWh),% Surconso,Période d'alerte,Catégorie
0,110,10.0,5.0,6,300.0,0.1,0,3.0
1,115,15.0,5.0,7,525.0,0.15,0,3.0
2,120,20.0,5.0,3,300.0,0.2,0,3.0
3,120,20.0,5.0,7,700.0,0.2,0,3.0
4,120,20.0,5.0,6,600.0,0.2,0,3.0
5,145,45.0,5.0,1,225.0,0.45,0,3.0
6,165,65.0,9.0,2,1170.0,0.65,1,1.0
7,175,75.0,9.0,1,675.0,0.75,1,1.0
8,200,100.0,9.0,1,900.0,1,1,1.0
9,85,5.0,7.0,1,35.0,0.0625,1,1.0


In [8]:
# Suppression des lignes avec des valeurs manquantes
df = df.dropna()

# Remplacer les valeurs manquantes par la médiane
df = df.replace('-', np.nan)  # Remplacez '-' par NaN
df = df.fillna(df.median())  # Remplacez NaN par la médiane de la colonne
df = df.astype(float)
# Séparation des données en ensembles d'entraînement et de test
X = df[["Nb d'heures", "Nb nuits/jours concernés", "% Surconso", "Période d'alerte"]]
y = df["Catégorie"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: convert into PyTorch Datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Step 4: translate into dataloader objects
batchsize    = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [9]:
# create a class for the model
def createTheMNISTNet(optimizerAlgo,learningrate):

  class mnistNet(nn.Module):
    def __init__(self):
      super().__init__()

      ### input layer
      self.input = nn.Linear(784,64)
      
      ### hidden layer
      self.fc1 = nn.Linear(64,32)
      self.fc2 = nn.Linear(32,32)

      ### output layer
      self.output = nn.Linear(32,10)

    # forward pass
    def forward(self,x):
      x = F.relu( self.input(x) )
      x = F.relu( self.fc1(x) )
      x = F.relu( self.fc2(x) )
      return self.output(x)
  
  # create the model instance
  net = mnistNet()
  
  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optifun = getattr( torch.optim,optimizerAlgo )
  optimizer = optifun(net.parameters(),lr=learningrate)

  return net,lossfun,optimizer

SVC()

In [10]:
# a function that trains the model

def funtion2trainTheModel(optimizerAlgo,learningrate):

  # number of epochs
  numepochs = 100
  
  # create a new model
  net,lossfun,optimizer = createTheMNISTNet(optimizerAlgo,learningrate)

  # initialize losses
  losses    = torch.zeros(numepochs)
  trainAcc  = []
  testAcc   = []


  # loop over epochs
  for epochi in range(numepochs):

    # loop over training data batches
    batchAcc  = []
    batchLoss = []
    for X,y in train_loader:

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      # compute accuracy
      matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
      matchesNumeric = matches.float()             # convert to numbers (0/1)
      accuracyPct = 100*torch.mean(matchesNumeric) # average and x100
      batchAcc.append( accuracyPct )               # add to list of accuracies
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # and get average losses across the batches
    losses[epochi] = np.mean(batchLoss)

    # test accuracy
    X,y = next(iter(test_loader)) # extract X,y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yHat = net(X)
      
    # compare the following really long line of code to the training accuracy lines
    testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

  # end epochs

  # function output
  return trainAcc,testAcc,losses,net

Accuracy on training set: 0.4896917738313942
Accuracy on test set: 0.4669387755102041


In [ ]:
# note: this experiment takes ~30 mins

# variables to loop over
learningRates = np.logspace(np.log10(.0001),np.log10(.1),6)
optimTypes = ['SGD','RMSprop','Adam']

# initialize performance matrix
finalPerformance = np.zeros((len(learningRates),len(optimTypes)))


# now for the experiment!
for idx_o,opto in enumerate(optimTypes):
  for idx_l,lr in enumerate(learningRates):
    trainAcc,testAcc,losses,net = funtion2trainTheModel(opto,lr)
    finalPerformance[idx_l,idx_o] = np.mean(testAcc[-10:])

In [ ]:
# plot the results! 
plt.plot(learningRates,finalPerformance,'o-',linewidth=2)
plt.legend(optimTypes)
plt.xscale('log')
plt.xlabel('Learning rates')
plt.ylabel('Test accuracy (ave. last 10 epochs)')
plt.title('Comparison of optimizers by learning rate')
plt.show()

In [12]:
# # Sauvegarder le modèle dans un fichier
# model_filename = "svm_model.pkl"
# joblib.dump(model_svm, model_filename)

['svm_model.pkl']

In [ ]:
#  import pandas as pd

#  df_ref = pd.read_excel('fichier2.xlsx', sheet_name='Feuil2', header=0, skiprows=0)
#  df_ref.to_csv('machine2.csv', index=False, encoding='utf-8-sig')